In [1]:
import pandas as pd
import itertools

In [2]:
df = pd.read_csv('../data/2_merged_da_data.csv')
impact_preds = pd.read_csv('../data/1_impact_predictions.csv')

driver_df = pd.read_csv('../data/1_driver_predictions.csv')
relevance_preds = pd.read_csv('../data/1_document_relevance.csv')
impact_drivers = impact_preds.merge(driver_df).merge(relevance_preds)

doc_continents = pd.read_csv('../data/doc_continents.csv').merge(pd.read_csv('../data/1_document_relevance.csv'), left_on="doc_id",right_on="id")

In [3]:
shp_ndf_df = pd.read_csv('../data/2_country_data.csv')

In [4]:
df_ndf = pd.read_csv(f'../data/study_gridcell_2.5.csv')
df_ndf = df_ndf.merge(df_ndf.groupby('doc_id').count().reset_index().rename(columns={"ndf_id":"gridcells"}))
df_ndf['weight'] = 1/df_ndf['gridcells']
df_ndf.head()

,ndf_id,doc_id,gridcells,weight
0,7674.0,3323227.0,6,0.166667
1,6804.0,3323227.0,6,0.166667
2,6720.0,3323227.0,6,0.166667
3,7327.0,3323227.0,6,0.166667
4,7674.0,3323227.0,6,0.166667


In [7]:
sector_cols = [x for x in impact_drivers.columns if "12 - " in x and "mean" in x]

continents = [
    "South America", 
    "North America", 
    "Africa", 
    "Europe", 
    "Asia",
    "Oceania"
]

In [9]:
sector_total = sector_cols + [" - Total - "]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[[x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total],["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(continents+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        ndf_ids = shp_ndf_df[shp_ndf_df['CONTINENT']==continent].ndf_id
        group = doc_continents[doc_continents["continent"]==continent] 
        cdf = impact_drivers[impact_drivers['id'].isin(group.doc_id)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 5
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 1
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems'), var)] = ci_area_sum(ndf, sub_ndf,thresh, var)       

table.to_html('../supplementary_data/area_pop_sums.md')
table.to_excel('../supplementary_data/area_pop_sums.xlsx')
                
table

(244, 6)
(244, 6)
(244, 6)
(244, 6)
(244, 6)
(244, 6)
(606, 6)
(606, 6)
(606, 6)
(606, 6)
(606, 6)
(606, 6)
(416, 6)
(416, 6)
(416, 6)
(416, 6)
(416, 6)
(416, 6)
(603, 6)
(603, 6)
(603, 6)
(603, 6)
(603, 6)
(603, 6)
(500, 6)
(500, 6)
(500, 6)
(500, 6)
(500, 6)
(500, 6)
(142, 6)
(142, 6)
(142, 6)
(142, 6)
(142, 6)
(142, 6)
(10368, 6)
(10368, 6)
(10368, 6)
(10368, 6)
(10368, 6)
(10368, 6)


Coastal and marine Ecosystems                 \
                                             area     population   
South America D&A                     8% (5%-31%)  26% (18%-51%)   
              Other                   3% (1%-27%)    6% (2%-15%)   
North America D&A                   33% (19%-50%)  58% (43%-70%)   
              Other                  11% (5%-12%)   27% (9%-27%)   
Africa        D&A                    12% (4%-23%)  22% (12%-37%)   
              Other                    2% (0%-4%)     1% (0%-4%)   
Europe        D&A                   16% (11%-21%)  59% (41%-71%)   
              Other                    0% (0%-0%)     0% (0%-0%)   
Asia          D&A                    19% (6%-29%)  47% (18%-58%)   
              Other                    3% (1%-4%)    9% (4%-13%)   
Oceania       D&A                   76% (72%-76%)  91% (59%-91%)   
              Other                 14% (14%-14%)     5% (5%-5%)   
Global        D&A                   22% (13%-33%)  44% (23%-57%)   
              Other                    5% (2%-8%)    8% (3%-11%)   

                    Human and managed systems                 \
                                         area     population   
South America D&A               26% (13%-62%)  55% (38%-79%)   
              Other              24% (5%-34%)   14% (9%-17%)   
North America D&A               40% (25%-57%)  64% (48%-72%)   
              Other               9% (7%-15%)  28% (26%-28%)   
Africa        D&A               32% (25%-46%)  68% (58%-79%)   
              Other               7% (3%-14%)     3% (3%-5%)   
Europe        D&A               25% (20%-33%)  80% (73%-88%)   
              Other                1% (0%-1%)     0% (0%-0%)   
Asia          D&A               61% (44%-78%)  80% (75%-82%)   
              Other                5% (5%-7%)  15% (15%-15%)   
Oceania       D&A               74% (53%-84%)  70% (54%-86%)   
              Other             14% (14%-14%)     5% (5%-5%)   
Global        D&A               38% (27%-53%)  73% (64%-79%)   
              Other               8% (4%-12%)  11% (11%-12%)   

                    Mountains, snow and ice                 \
                                       area     population   
South America D&A              11% (8%-15%)  19% (14%-32%)   
              Other            13% (6%-14%)    6% (3%-10%)   
North America D&A             34% (18%-54%)  43% (27%-56%)   
              Other             8% (5%-19%)   19% (8%-24%)   
Africa        D&A                3% (1%-9%)    7% (6%-14%)   
              Other              0% (0%-1%)     0% (0%-1%)   
Europe        D&A              21% (8%-30%)  58% (25%-79%)   
              Other              0% (0%-1%)     0% (0%-0%)   
Asia          D&A             27% (14%-43%)  39% (12%-56%)   
              Other              4% (2%-4%)   14% (6%-14%)   
Oceania       D&A                3% (2%-8%)  37% (15%-53%)   
              Other              1% (1%-3%)     4% (4%-4%)   
Global        D&A              17% (9%-27%)  32% (13%-46%)   
              Other              4% (2%-7%)    9% (4%-10%)   

                    Rivers, lakes, and soil moisture                 \
                                                area     population   
South America D&A                       16% (6%-45%)  33% (14%-68%)   
              Other                     19% (3%-34%)   15% (5%-19%)   
North America D&A                      52% (24%-64%)  60% (41%-69%)   
              Other                     15% (9%-18%)  28% (28%-28%)   
Africa        D&A                       17% (5%-35%)  44% (18%-68%)   
              Other                      9% (1%-13%)     4% (2%-7%)   
Europe        D&A                      24% (18%-31%)  77% (64%-86%)   
              Other                       0% (0%-0%)     0% (0%-0%)   
Asia          D&A                      61% (34%-72%)  75% (50%-81%)   
              Other                       5% (5%-5%)  15% (15%-15%)   
Oceania       D&A                       19% (4%-83%)  53% (43

In [19]:
sector_total = sector_cols + [" - Total - "]

incs = list(shp_ndf_df['Income category'].unique())

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[[x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total],["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[incs + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%} ({results[1]:.0%}-{results[2]:.0%})"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(incs+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        group = shp_ndf_df[shp_ndf_df['Income category']==continent]
        ndf_ids = group.ndf_id
        doc_ids = df_ndf[df_ndf['ndf_id'].isin(ndf_ids)].doc_id
        cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 5
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 1
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems'), var)] = ci_area_sum(ndf, sub_ndf,thresh, var)       

table.to_html('../supplementary_data/area_pop_sums_income.md')
table.to_excel('../supplementary_data/area_pop_sums_income.xlsx')
                
table

Coastal and marine Ecosystems                 \
                                                      area     population   
5. Low income          D&A                     8% (2%-20%)   17% (5%-27%)   
                       Other                    3% (0%-5%)  12% (11%-17%)   
4. Lower middle income D&A                    26% (9%-37%)  59% (19%-65%)   
                       Other                    5% (1%-8%)   10% (2%-16%)   
3. Upper middle income D&A                    11% (5%-23%)  29% (17%-49%)   
                       Other                    2% (1%-9%)     4% (1%-6%)   
1. High Income         D&A                   45% (34%-56%)  77% (60%-85%)   
                       Other                   9% (6%-10%)     9% (6%-9%)   
Global                 D&A                   22% (13%-33%)  44% (23%-57%)   
                       Other                    5% (2%-8%)    8% (3%-11%)   

                             Human and managed systems                 \
                                                  area     population   
5. Low income          D&A               35% (29%-45%)  53% (46%-61%)   
                       Other               5% (4%-11%)  20% (20%-21%)   
4. Lower middle income D&A               50% (42%-74%)  77% (71%-80%)   
                       Other              14% (9%-19%)  18% (18%-18%)   
3. Upper middle income D&A               34% (21%-51%)  80% (69%-88%)   
                       Other               9% (3%-11%)     6% (4%-6%)   
1. High Income         D&A               50% (37%-66%)  85% (80%-90%)   
                       Other               8% (6%-13%)     9% (8%-9%)   
Global                 D&A               38% (27%-53%)  73% (64%-79%)   
                       Other               8% (4%-12%)  11% (11%-12%)   

                             Mountains, snow and ice                 \
                                                area     population   
5. Low income          D&A               6% (3%-11%)    6% (3%-17%)   
                       Other              1% (1%-2%)   16% (5%-17%)   
4. Lower middle income D&A              22% (6%-29%)  49% (12%-58%)   
                       Other            12% (3%-12%)   17% (7%-17%)   
3. Upper middle income D&A              13% (9%-24%)  19% (11%-42%)   
                       Other              2% (2%-3%)     2% (0%-4%)   
1. High Income         D&A             32% (16%-48%)  68% (37%-82%)   
                       Other             5% (3%-13%)     7% (6%-8%)   
Global                 D&A              17% (9%-27%)  32% (13%-46%)   
                       Other              4% (2%-7%)    9% (4%-10%)   

                             Rivers, lakes, and soil moisture                 \
                                                         area     population   
5. Low income          D&A                       22% (7%-37%)  36% (11%-54%)   
                       Other                       4% (2%-8%)  20% (18%-23%)   
4. Lower middle income D&A                      38% (19%-59%)  67% (37%-77%)   
                       Other                     13% (8%-20%)  18% (17%-19%)   
3. Upper middle income D&A                      31% (16%-47%)  73% (52%-85%)   
                       Other                      9% (2%-12%)     6% (4%-6%)   
1. High Income         D&A                      46% (24%-68%)  81% (71%-86%)   
                       Other                     12% (5%-14%)     9% (8%-9%)   
Global                 D&A                      32% (16%-49%)  62% (40%-75%)   
                       Other                      9% (3%-12%)  11% (10%-12%)   

                             Terrestrial ecosystems                 \
                                               area     population   
5. Low income          D&A            37% (25%-48%)  51% (41%-61%)   
                       Other            7% (3%-19%)  21% (18%-26%)   
4. Lower middle income D&A            64% (41%-71%)  78% (58%-80%)   
                       Other          19% (14%-21%)  19% (17%-1

In [11]:
sector_total = sector_cols + [" - Total - "]

colindex = pd.MultiIndex.from_tuples(list(itertools.product(*[[x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total],["area","population"]])))
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=colindex)

def ci_area_sum(ndf, sub_ndf, threshold, var):
    results = []
    if var=="area":
        ndf = ndf[ndf['is_land']==True]
        sub_ndf = sub_ndf[sub_ndf['is_land']==True]
        
    ndf_sum = ndf[var].sum()
    for suffix in ["","_lower","_upper"]:
        results.append(sub_ndf[sub_ndf[f"total_study_prop{suffix}"]>=threshold][var].sum()/ndf_sum)
        
    return f"{results[0]:.0%}"

def ci_df(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return [mid_df.id, lower_df.id, upper_df.id]

for i, continent in enumerate(continents+["Global"]):
    if continent=="Global":
        ndf_ids = df['index']
        cdf = impact_drivers
    else:
        ndf_ids = shp_ndf_df[shp_ndf_df['CONTINENT']==continent].ndf_id
        group = doc_continents[doc_continents["continent"]==continent] 
        cdf = impact_drivers[impact_drivers['id'].isin(group.doc_id)]
    for k, sector in enumerate(sector_total):
        if sector==" - Total - ":
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction'])
            thresh = 5
        else:
            mid_ids, lower_ids, upper_ids = ci_df(cdf,['0 - relevance - mean_prediction', sector])
            thresh = 1
        id_subsets = [mid_ids, upper_ids, lower_ids]
        ndf = df[df['index'].isin(ndf_ids)][['index','study_addition','is_land','n_da_impacts','area','population']]
        for var, driver in [("temp","Temperature"),("precip","Precipitation")]:
            for l, (label, suffix) in enumerate([(f"{var}_study_prop","mean_prediction"),(f"{var}_study_prop_upper","upper_pred"),(f"{var}_study_prop_lower","lower_pred")]):
                var_studies = driver_df[(driver_df[f'6 - {driver} - {suffix}']>=0.5) & (driver_df['id'].isin(id_subsets[l]))].id
                var_ndf = df_ndf[df_ndf['doc_id'].isin(var_studies)].groupby('ndf_id')['weight'].sum().to_frame().rename(columns={"weight":label})
                ndf = ndf.merge(var_ndf, left_on="index", right_on="ndf_id", how="left")
                ndf[label] = ndf[label].fillna(0) 
                
        for suffix in ["","_lower","_upper"]:
                
            ndf.loc[ndf['study_addition']=="temp",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="temp",f"temp_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="precip",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="precip",f"precip_study_prop{suffix}"]
            ndf.loc[ndf['study_addition']=="add",f"total_study_prop{suffix}"] = ndf.loc[ndf['study_addition']=="add",[f"precip_study_prop{suffix}",f"temp_study_prop{suffix}"]].sum(axis=1)            
            ndf[f"total_study_prop{suffix}"].fillna(0)           
        
        for j, da in enumerate(["D&A","Other"]):
            if da=="D&A":
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])>0)]
            else:
                sub_ndf = ndf[(abs(ndf['n_da_impacts'])==0)]
                
            for var in ["area","population"]:
                table.loc[(continent,da),(sector.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems'), var)] = ci_area_sum(ndf, sub_ndf,0, var)       

table.to_html('../supplementary_data/area_pop_sums_all.md')
table.to_excel('../supplementary_data/area_pop_sums_all.xlsx')
                
table

Coastal and marine Ecosystems             \
                                             area population   
South America D&A                             64%        81%   
              Other                           36%        19%   
North America D&A                             71%        72%   
              Other                           29%        28%   
Africa        D&A                             73%        89%   
              Other                           27%        11%   
Europe        D&A                             95%       100%   
              Other                            5%         0%   
Asia          D&A                             93%        85%   
              Other                            7%        15%   
Oceania       D&A                             86%        95%   
              Other                           14%         5%   
Global        D&A                             80%        87%   
              Other                           20%        13%   

                    Human and managed systems             \
                                         area population   
South America D&A                         64%        81%   
              Other                       36%        19%   
North America D&A                         71%        72%   
              Other                       29%        28%   
Africa        D&A                         73%        89%   
              Other                       27%        11%   
Europe        D&A                         95%       100%   
              Other                        5%         0%   
Asia          D&A                         93%        85%   
              Other                        7%        15%   
Oceania       D&A                         86%        95%   
              Other                       14%         5%   
Global        D&A                         80%        87%   
              Other                       20%        13%   

                    Mountains, snow and ice             \
                                       area population   
South America D&A                       64%        81%   
              Other                     36%        19%   
North America D&A                       71%        72%   
              Other                     29%        28%   
Africa        D&A                       73%        89%   
              Other                     27%        11%   
Europe        D&A                       95%       100%   
              Other                      5%         0%   
Asia          D&A                       93%        85%   
              Other                      7%        15%   
Oceania       D&A                       86%        95%   
              Other                     14%         5%   
Global        D&A                       80%        87%   
              Other                     20%        13%   

                    Rivers, lakes, and soil moisture             \
                                                area population   
South America D&A                                64%        81%   
              Other                              36%        19%   
North America D&A                                71%        72%   
              Other                              29%        28%   
Africa        D&A                                73%        89%   
              Other                              27%        11%   
Europe        D&A                                95%       100%   
              Other                               5%         0%   
Asia          D&A                                93%        85%   
              Other                               7%        15%   
Oceania       D&A                                86%        95%   
              Other                              14%         5%   
Global        D&A                                80%        87%   
              Other                              20%        13%   

                    Terrestrial 

In [26]:
glob_index = pd.MultiIndex.from_tuples(list(itertools.product(*[continents + ["Global"],["D&A","Other"]])))
table = pd.DataFrame(index=glob_index,columns=[x.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems') for x in sector_total])

df_studies = pd.read_csv('../data/study_da.csv')

da_studies = df_studies[df_studies[['precip_da_prop','temp_da_prop']].max(axis=1)>0]

def ci(df, var_list):
    lower_df = df
    mid_df = df
    upper_df = df
    for var in var_list:
        mid_df = mid_df[mid_df[var]>=0.5]
        upper_df = upper_df[upper_df[var.replace("mean_prediction","upper_pred")]>=0.5]
        lower_df = lower_df[lower_df[var.replace("mean_prediction","lower_pred")]>=0.5]
        
    return f"{mid_df.shape[0]} ({lower_df.shape[0]}-{upper_df.shape[0]})"

for i, continent in enumerate(continents+["Global"]):
    if continent=="Global":
        doc_ids = impact_drivers.id
    else:
        doc_ids = doc_continents[doc_continents["continent"]==continent].doc_id
    cdf = impact_drivers[impact_drivers['id'].isin(doc_ids)]
    dadfs = [cdf[cdf['id'].isin(da_studies.id)],cdf[~cdf['id'].isin(da_studies.id)]]
    for j, da in enumerate(["D&A","Other"]):
        cdf = dadfs[j]
        for k, sector in enumerate(sector_total):
            if "Total" in sector:
                table.loc[(continent,da),sector.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems')] = ci(cdf,['0 - relevance - mean_prediction'])
            else:
                table.loc[(continent,da),sector.split(' - ')[1].replace('managed','managed systems').replace('ES','ecosystems')] = ci(cdf,['0 - relevance - mean_prediction',sector])
            
table.to_html('../supplementary_data/continent_sector_n_studies.md')
table.to_excel('../supplementary_data/continent_sector_n_studies.xlsx')
table

Coastal and marine Ecosystems Human and managed systems  \
South America D&A                   365 (194-662)             460 (275-925)   
              Other                  209 (74-520)              130 (61-322)   
North America D&A                2429 (1304-4241)           1708 (936-3300)   
              Other               1620 (579-3859)            446 (180-1240)   
Africa        D&A                   448 (219-881)           1102 (625-2039)   
              Other                 345 (105-865)             364 (164-956)   
Europe        D&A                 1300 (628-2426)           1637 (934-3162)   
              Other                678 (222-1900)              162 (70-620)   
Asia          D&A                 1164 (604-2097)          3516 (2124-6329)   
              Other                712 (256-1856)            917 (410-2383)   
Oceania       D&A                  890 (499-1484)             494 (271-979)   
              Other                594 (235-1239)              117 (50-367)   
Global        D&A                5529 (2957-9755)         7729 (4487-14423)   
              Other             7078 (2838-15628)         4457 (2036-11717)   

                    Mountains, snow and ice Rivers, lakes, and soil moisture  \
South America D&A             296 (184-491)                    384 (207-794)   
              Other             57 (35-116)                    234 (141-469)   
North America D&A           1734 (972-3203)                 2621 (1415-4899)   
              Other          608 (295-1259)                  1715 (876-3595)   
Africa        D&A             268 (143-514)                   747 (393-1422)   
              Other             79 (44-199)                   531 (259-1077)   
Europe        D&A            985 (547-1937)                  1307 (743-2638)   
              Other           344 (198-723)                   834 (439-1850)   
Asia          D&A           1105 (700-2011)                 2968 (1790-5292)   
              Other           275 (144-531)                 1987 (1138-3699)   
Oceania       D&A             349 (191-622)                    521 (296-993)   
              Other             79 (44-192)                    272 (125-568)   
Global        D&A          3783 (2201-7050)                6623 (3777-12384)   
              Other        2524 (1325-5178)                6182 (3278-12777)   

                    Terrestrial ecosystems                Total  
South America D&A          1366 (699-2681)     3674 (2400-5760)  
              Other           235 (94-592)      1061 (605-1995)  
North America D&A        7835 (4308-13552)  21745 (14364-31884)  
              Other       2259 (1029-4821)    8868 (5002-15196)  
Africa        D&A          1556 (706-2951)     5323 (3391-8104)  
              Other         447 (182-1036)     2251 (1220-4105)  
Europe        D&A        6006 (3389-10396)   13991 (9105-21466)  
              Other         816 (361-2020)     3762 (2089-7232)  
Asia          D&A        5822 (3270-10040)  20885 (14705-29783)  
              Other         991 (402-2269)    6764 (3999-11548)  
Oceania       D&A          1539 (777-2866)     5482 (3579-8202)  
              Other          355 (155-873)     1922 (1047-3441)  
Global        D&A      19622 (10796-34361)  57366 (38371-85227)  
              Other     15366 (7724-31305)  44373 (25594-78626)